# Notebook 8: Particle Swarms

<!--
<div style="float: right; width: 50%; padding-left:10px;">
<img src="media/CompositeImage.png" width=100%>
<caption>
<i>
    Flow in a pipe with inflow at the left boundary
    after 50, 100, 150 timesteps (top to bottom) showing the
    progression of the impulsive initial condition. For details,
    see the notebook code.
</i>
</caption>
</div>
-->

We used a particle swarm to track the flow in Example 7. We called this a "passive" swarm because the points did not influence the flow in any way but was simply carried along. 

Particle swarms are unstructured data objects that live within the computational domain. Their points can be moved arbitrarily through the domain and points may migrate from one process to another when the coordinates are changed. By default they carry only the particle location, but we can add scalar, vector and tensor variables to the swarm and they will be transported with the particles.

Particle transport is usually through a velocity or displacement field that incrementally changes the locations. This is a common use, but particles can be used to represent any unstructured field. For example, during mesh adaptation, the nodal points from the previous mesh are equivalent to a disconnected swarm from the point of view of the new mesh. The same is true when reading data save from one mesh to the `MeshVariables` on another.


In [1]:
#|  echo: false  # Hide in html version

# This is required to fix pyvista 
# (visualisation) crashes in interactive notebooks (including on binder)

import nest_asyncio
nest_asyncio.apply()

In [2]:
#| output: false # Suppress warnings in html version

import underworld3 as uw
import numpy as np
import sympy

[esdhcp-137.anu.edu.au:61998] shmem: mmap: an error occurred while determining whether or not /var/folders/tx/95gr762j29z4tt5d1dnqlgth0000gn/T//ompi.esdhcp-137.501/jf.0/2846228480/sm_segment.esdhcp-137.501.a9a60000.0 could be created.


In [3]:
res = 12
width = 8

mesh = uw.meshing.UnstructuredSimplexBox(
    cellSize=1/res,
    minCoords=(0.0,0.0),
    maxCoords=(width, 1.0),
    qdegree=3,     
)

# Coordinate directions etc
x, y = mesh.CoordinateSystem.X


In [4]:
# original y coordinate

y0 = uw.discretisation.MeshVariable("Y0", 
                                    mesh, 
                                    vtype=uw.VarType.SCALAR,
                                    varsymbol=r"y_0")

with mesh.access(y0):
    y0.data[:,0] = uw.function.evalf(y, y0.coords)

#### Deform the mesh

Move all nodes down to meet an undulating lower surface. The
displacement field is smooth and continuous, so there is no 
particular need to remesh in this case. However, it is generally
better to consider either deforming the mesh with `gmsh` before
triangulation, or remeshing (both are possible with underworld3, 
but not as simple to demonstrate).

In [5]:
new_coords = mesh.data
dy =  (1-y) * (x/16 + sympy.sin(sympy.pi * x)/10) 
new_coords[:,1] = uw.function.evalf(y-dy, mesh.data)

display(dy)

mesh.deform_mesh(new_coords)


(1 - N.y)*(N.x/16 + sin(N.x*pi)/10)

In [6]:
kdxy = mesh._index.kdtree_points()

In [7]:
mesh._build_kd_tree_index()
kdxy1 = mesh._index.kdtree_points()

In [8]:
kdxy[:,1].max()

0.9995226007528144

In [9]:
kdxy1[:,1].min()

-0.5066318954971105

In [10]:
if uw.mpi.size == 1:
    import pyvista as pv
    import underworld3.visualisation as vis

    pvmesh = vis.mesh_to_pv_mesh(mesh)
    pvmesh.point_data["y0"] = vis.scalar_fn_to_pv_points(pvmesh, y0.sym)
    
    pl = pv.Plotter(window_size=(750, 250))


    pl.add_mesh(
        pvmesh,
        cmap="RdBu_r",
        edge_color="Grey",
        edge_opacity=0.33,
        show_edges=True,
        use_transparency=False,
        opacity=0.75,
        show_scalar_bar=False,
    )

    pl.show(jupyter_backend="html")

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

### Add a swarm to this mesh

A swarm object can be constructed either by adding local points (Example 7) or by filling the mesh with a given density. The density value that we provide (`fill_param`) adds particles on the Gaussian integration points: 0 will populate the centroids of the mesh elements A value of 1 provides three points per triangle, four per quad, four in a tetrahedron, eight in a hexahedron (the points that support linear interpolation in standard FEM).




In [11]:
swarm = uw.swarm.Swarm(mesh)
swarm.add_particles_with_coordinates(mesh._get_mesh_centroids())
# swarm.populate_petsc(0)


2725

In [12]:
if uw.mpi.size == 1:
    import pyvista as pv
    import underworld3.visualisation as vis

    pvmesh = vis.mesh_to_pv_mesh(mesh)
    pvmesh.point_data["y0"] = vis.scalar_fn_to_pv_points(pvmesh, y0.sym)

    swarm_points = vis.swarm_to_pv_cloud(swarm)
    
    pl = pv.Plotter(window_size=(750, 250))


    pl.add_mesh(
        pvmesh,
        cmap="RdBu_r",
        edge_color="Grey",
        edge_opacity=0.33,
        show_edges=True,
        use_transparency=False,
        opacity=0.75,
        show_scalar_bar=False,
    )

    pl.add_points(swarm_points.points, 
                  point_size=5,
                  style='points',
                  color="Black", 
                  opacity=1)
    
    pl.camera.zoom(3)

    pl.show(jupyter_backend="trame")

Widget(value='<iframe src="http://localhost:54386/index.html?ui=P_0x1599f1650_1&reconnect=auto" class="pyvista…

In [13]:
vec = swarm.celldm.getCoordinates().array.reshape(-1,2)

In [14]:
vec[:,1].min()

-0.5080509159623103

In [15]:
0/0

ZeroDivisionError: division by zero

In [ ]:
# Mesh variables for the unknowns

v_soln = uw.discretisation.MeshVariable("V0", mesh, 2, degree=2, varsymbol=r"{v_0}")
p_soln = uw.discretisation.MeshVariable("p", mesh, 1, degree=1, continuous=True)
vorticity  = uw.discretisation.MeshVariable("omega", mesh, 1, degree=1, continuous=True, varsymbol=r"\omega")

In [ ]:

navier_stokes = uw.systems.NavierStokes(
    mesh, 
    velocityField=v_soln, 
    pressureField=p_soln, 
    solver_name="Navier_stokes",
    rho=reynolds_number,
    order=1,
)

navier_stokes.constitutive_model = uw.constitutive_models.ViscousFlowModel
navier_stokes.constitutive_model.Parameters.shear_viscosity_0 = 1
navier_stokes.tolerance = 1.0e-3

navier_stokes.petsc_options["fieldsplit_velocity_mg_coarse_pc_type"] = "svd"

navier_stokes.bodyforce = sympy.Matrix((0,0))

# Inflow boundary - incoming jet
navier_stokes.add_essential_bc(((4*y*(1-y))**8, 0), "Left")
navier_stokes.add_essential_bc((0, 0), "Bottom")
navier_stokes.add_essential_bc((0, 0), "Top")


In [ ]:
vorticity_from_v = uw.systems.Projection(mesh, vorticity)
vorticity_from_v.uw_function = mesh.vector.curl(v_soln.sym)
vorticity_from_v.smoothing = 1.0e-3
# nodal_vorticity_from_v.petsc_options.delValue("ksp_monitor")

In [ ]:
passive_swarm = uw.swarm.Swarm(mesh=mesh)
passive_swarm.populate(
    fill_param=1,
)

# add new points at the inflow
npoints = 100
passive_swarm.dm.addNPoints(npoints)
with passive_swarm.access(passive_swarm.particle_coordinates):
    for i in range(npoints):
        passive_swarm.particle_coordinates.data[-1 : -(npoints + 1) : -1, :] = np.array(
            [0.0, 0.25] + 0.5 * np.random.random((npoints, 2))
        )


In [ ]:
navier_stokes.solve(timestep=0.01)
vorticity_from_v.solve()

In [ ]:
# Keep the initialisation separate
# so we can run the loop below again without resetting
# the timer.

max_steps = 50
timestep = 0
elapsed_time = 0.0
delta_t = 0.05

In [ ]:


for step in range(0, max_steps):

    navier_stokes.solve(zero_init_guess=False, timestep=delta_t)

    passive_swarm.advection(v_soln.sym, delta_t, order=2, corrector=False, evalf=True)

    new_points = 100
    new_coords = np.array([0.0, 0.25] + 0.5 * np.random.random((new_points, 2)))
    passive_swarm.add_particles_with_coordinates(new_coords)    
    
    # Save the data at every 10th step

    if timestep % 10 == 0:
        vorticity_from_v.solve()        

        mesh.write_timestep(
            "Example_7",
            meshUpdates=True,
            meshVars=[p_soln, v_soln, vorticity],
            outputPath="Example_output",
            index=timestep,
        )

        passive_swarm.write_timestep(
            "Example_7",
            "passive_swarm",
            swarmVars=None,
            outputPath="Example_output",
            index=timestep,
            force_sequential=True,
        )


    
    timestep += 1
    elapsed_time += delta_t

    print(f"Timestep: {timestep}, time {elapsed_time:.4f}")



In [ ]:
# visualise it


if uw.mpi.size == 1:
    import pyvista as pv
    import underworld3.visualisation as vis

    pvmesh = vis.mesh_to_pv_mesh(mesh)
    pvmesh.point_data["P"] = vis.scalar_fn_to_pv_points(pvmesh, p_soln.sym)
    pvmesh.point_data["Omega"] = vis.scalar_fn_to_pv_points(pvmesh, vorticity.sym)
    pvmesh.point_data["V"] = vis.vector_fn_to_pv_points(pvmesh, v_soln.sym)

    pvmesh_v = vis.meshVariable_to_pv_mesh_object(v_soln, alpha=None)
    pvmesh_v.point_data["V"] = vis.vector_fn_to_pv_points(pvmesh_v, v_soln.sym)

    skip = 1
    points = np.zeros((mesh._centroids[::skip].shape[0], 3))
    points[:, 0] = mesh._centroids[::skip, 0]
    points[:, 1] = mesh._centroids[::skip, 1]
    point_cloud = pv.PolyData(points)

    pvstream = pvmesh.streamlines_from_source(
        point_cloud, vectors="V", 
        integration_direction="both", 
        integrator_type=45,
        surface_streamlines=True,
        initial_step_length=0.01,
        max_time=1.0,
        max_steps=500, 
    )

    passive_swarm_points = uw.visualisation.swarm_to_pv_cloud(passive_swarm)

    pl = pv.Plotter(window_size=(750, 750))

    pl.add_mesh(
        pvmesh,
        cmap="RdBu_r",
        edge_color="Grey",
        edge_opacity=0.33,
        scalars="Omega",
        show_edges=True,
        use_transparency=False,
        opacity=0.75,
        show_scalar_bar=False,
    )

    # Optional: plot streamlines
    # pl.add_mesh(pvstream, opacity=0.3, show_scalar_bar=False, cmap="Greys_r", render_lines_as_tubes=False)

    pl.add_points(
            passive_swarm_points,
            color="Black",
            render_points_as_spheres=False,
            point_size=4,
            opacity=0.33,
        )

    # pl.add_arrows(pvmesh_v.points, pvmesh_v.point_data["V"], mag=0.1)

    #pl.camera_position = 'xy'
    pl.camera.position = (2.0, 0.5, 4)
    
    pl.export_html(f"html5/ns_flow_plot_{timestep}.html")

    pl.screenshot(f"ns_flow_at_{timestep}.png", window_size=(4000,1000), return_img=False)


In [ ]:
#| fig-cap: "Interactive Image: Convection model output"
from IPython.display import IFrame
IFrame(src=f"html5/ns_flow_plot_{timestep}.html", width=1000, height=400)

## Exercise - 


